In [1]:
# Loading the required package 
import cobra 

In [2]:
# Loading Model
model = cobra.io.load_matlab_model("simple_model.mat")

In [3]:
# Analysing the model features
print("The model has {} reactions".format(len(model.reactions)))
print("The model has {} metabolites".format(len(model.metabolites)))

The model has 5 reactions
The model has 4 metabolites


In a paper you would report the degree of the model you are working with, to give an idea of how complex the model is. 

The degree of a model is equal to the total # of reactions - the # of input & output reactions. 

The model we are working with has a degree of 1 (our photosynthesis reaction). This is the simplest model possible. 

In [4]:
# We can print all of the model reactions using a for loop
for r in model.reactions:
    print(r)
#     # We can check if the reactions are mass balanced
#     print(r.check_mass_balance())
#     # We can check the upper and lower bounds 
#     print(r.bounds)
    print(" ")

CO2_In:  --> CO2_h
 
H2O_In:  --> H2O_h
 
O2_Out: O2_h --> 
 
Suc_Out: Suc_h --> 
 
Photo: 6.0 CO2_h + 6.0 H2O_h --> 6.0 O2_h + Suc_h
 


In [5]:
# Setting an objective function 
model.objective = model.reactions.get_by_id("Suc_Out")
print(model.objective)

Maximize
-1.0*Suc_Out_reverse_77f6c + 1.0*Suc_Out


Setting `Suc_Out` as the objective function is an optimization constraint. 

We can now calculate model solutions that fulfil this constraints (i.e. produce the maximum possible sugar export).

In [6]:
# Running a Flux Balance Analysis
solution = model.optimize()
print(solution)

# Print single Flux Solutions 
model.summary()

<Solution 166.667 at 0x7f777a251c90>
IN FLUXES     OUT FLUXES      OBJECTIVES
------------  --------------  ------------
CO2_h  1e+03  O2_h     1e+03  Suc_Out  167
H2O_h  1e+03  Suc_h  167


The maximum possible flux through `Suc_Out` is 167. 

This is because the maximum CO2 and H2O input is 1000.

We need 6 carbons for every sugar, and 1000/6 = 166.666666 (note the rounding). 

In [7]:
# We can impose constraints onto our model by setting upper and lower bounds
r = model.reactions.get_by_id("CO2_In")
r.lower_bound = 400.0
r.upper_bound = 600.0

In [8]:
# Re-running a Flux Balance Analysis
solution = model.optimize()
print(solution)

# Print single Flux Solutions 
model.summary()

<Solution 100.000 at 0x7f777a203810>
IN FLUXES    OUT FLUXES    OBJECTIVES
-----------  ------------  ------------
CO2_h  600   O2_h   600    Suc_Out  100
H2O_h  600   Suc_h  100


In [9]:
r = model.reactions.get_by_id("CO2_In")
r.lower_bound = 400.0
r.upper_bound = 600.0

Here we said that the maximum CO2 input is a flux of 600, so the maximum output of sugar is 100.

It is also possible to calculate the feasible upper and lower bounds, rather than the single flux solutions.

This is called a flux variability analysis (FVA).

Because our model is small and each reaction is dependent on all reactions in the model, the upper and lower bounds are always the same. 

But you can imagine that in a bigger model there might be many different ways to get the same solution! That's when FVA becomes useful. 

In [10]:
FVAresults = cobra.flux_analysis.flux_variability_analysis(model,model.reactions)
print(FVAresults)

         maximum  minimum
CO2_In     600.0    600.0
H2O_In     600.0    600.0
O2_Out     600.0    600.0
Suc_Out    100.0    100.0
Photo      100.0    100.0
